# Retrieval-Augmented Generation (RAG) for German Documents — Tutorial


A small production-style RAG system that:
1) ingests German PDFs 
2) cleans/splits text into semantically sized chunks,
3) embeds those chunks with a multilingual model,
4) indexes them in FAISS for fast vector search, and
5) answers questions by retrieving top-k passages and prompting an LLM.


### Folder Layout:

repo/

├─ pdfs/                      #  German PDFs

├─ Faiss_index_german/        # FAISS index will be stored here

└─ RAG.ipynb                  # the notebook this tutorial explains

(If your index folder has a different name, update the paths in the code)

## 0) Prerequisites

Installing required libraries

In [1]:
!pip install transformers sentence-transformers langchain-community langchain torch faiss-cpu numpy pypdf einops unstructured --quiet

In [1]:
!nvidia-smi

Wed Jul 30 20:27:25 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:05.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      2MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Importing Necessary Libraries

import numpy as np
import torch
import tqdm

# LangChain building blocks
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

#Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings



## 2) PDF Loading

Iterating over the the pdf folder and reading them

In [ ]:
#Collecting paths of all the pdf

import os 

pdfs = []
for dirpath, _, filenames in os.walk("/teamspace/studios/this_studio/RAG/data"):
    for file in filenames:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(dirpath, file))
len(pdfs)

166

In [6]:
def load_and_split_pdf(pdf_path, text_splitter):
    loader = PyPDFLoader(pdf_path)
    docs_before_split = loader.load()
    docs_after_split = text_splitter.split_documents(docs_before_split)
    for doc in docs_after_split:
        doc.metadata["source"] = pdf_path
    return docs_after_split


## 3) Chunking

Aim: ~500 chars per chunk with ~10% overlap 

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
    )

## 4) Embeddings (multilingual for German)

default model: jinaai/jina-embeddings-v3
This model has the best multilingual performance. HF_Link:

In [9]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="jinaai/jina-embeddings-v3", 
    # model_name="intfloat/multilingual-e5-large-instruct",
    model_kwargs={'device':device,
    "trust_remote_code": True}, 
    encode_kwargs={'normalize_embeddings': True}
)


## 5) FAISS Indexing 

Why FAISS: it’s fast, mature, and file-backed — perfect for notebooks and small servers.


In [ ]:
for path in tqdm.tqdm(pdfs):
    split_docs = load_and_split_pdf(path, text_splitter)

    vectorstore = FAISS.from_documents(split_docs, huggingface_embeddings)


100%|██████████| 166/166 [38:19<00:00, 13.85s/it]


In [ ]:
#Saving the index to local system

# vectorstore.save_local("all_pdf_faiss_Index")

NameError: name 'vectorstore' is not defined

In [ ]:
# Sample query:

sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)


In [ ]:
# vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)
# vectorstore.save_local("Faiss_index_german")

# # vectorstore=FAISS.load_local("Faiss_index",huggingface_embeddings,allow_dangerous_deserialization=True)

In [21]:
vectorstore=FAISS.load_local("all_pdf_faiss_Index",huggingface_embeddings,allow_dangerous_deserialization=True)

### Testing retriever

In [23]:
query = """Security objectives for the TOE"""  
relevant_documents = vectorstore.similarity_search(query)
print(f'Total Documents retrieved: {len(relevant_documents)} \n ---First one:---\n')
print(relevant_documents[0].page_content)


Total Documents retrieved: 4 
 ---First one:---

ständen eigene getroffen werden. Hierzu gehören beispielsweise eine Zugriffsbeschränkung 
für betroffene Dienste (z. B. Zugriffsmöglichkeiten nur noch aus dem Intranet bzw. über VPN), 
striktere Firewall-Regelungen, der Einsatz von oder eine Erweiterung bestehender Paketfilter 
oder Monitoring-Tools sowie der Einsatz generischer Mitigation-Tools.
 4.4 Produktverzicht
Im schlimmsten Fall ist es aufgrund einer akuten Gefährdungslage nicht mehr vertretbar, ein


In [24]:
relevant_documents[0]

Document(id='498d9e55-b60a-4095-b6e9-2fabe8094972', metadata={'producer': 'LibreOffice 5.2', 'creator': 'Writer', 'creationdate': '2018-06-22T11:54:35+02:00', 'author': 'Bundesamt für Sicherheit in der Informationstechnik', 'keywords': '"Software, Schwachstellen, Warndienste, Gegenmaßnahmen, Softwareverzicht"', 'moddate': '2018-06-26T11:01:58+02:00', 'subject': 'BSI-Veröffentlichungen zur Cyber-Sicherheit', 'title': 'Sicheres Schwachstellen- und Patch-Management - Empfehlungen für kleine Unternehmen und Selbstständige', 'source': '/teamspace/studios/this_studio/RAG/data/002-Cyber security recommendations by attack targets/008-Companies - general/002-Step 2 Implement some initial safeguards/BSI-CS_093.pdf', 'total_pages': 5, 'page': 2, 'page_label': '3'}, page_content='ständen eigene getroffen werden. Hierzu gehören beispielsweise eine Zugriffsbeschränkung \nfür betroffene Dienste (z. B. Zugriffsmöglichkeiten nur noch aus dem Intranet bzw. über VPN), \nstriktere Firewall-Regelungen, der

In [27]:
print(relevant_documents[2].page_content)

stellen und Sicherheitsupdates in kleinen IT-Umgebungen dienen.
Name Ver-
sion
Gefähr-
dungs-
potenzial
Wichtigkeit Offene 
Schwach-
stellen?
Gefährdung Sicherheits-
update 
verfügbar
Sicherheits
-update 
eingespielt
Mitigation letzte 
Prüfung
Anmerkungen
TextEditor 1.31 niedrig niedrig nein - - 17.11.2013 -
Browser A 31 hoch hoch ja hoch nein nein Browser B 
nutzen
18.11.2013 CVE 2013-xxxx, 
Remote Code 
Execution mit 
Benutzerrechten


In [15]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

### 6) Loading LLM: llama3.2- 1B

In [ ]:
## logging in HF to access gated models

from huggingface_hub import login
from tokens import HF_TOKEN  # add your tokens in tokens.py file
login(token=HF_TOKEN)


In [17]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="meta-llama/Llama-3.2-1B-Instruct",
    device=0, # CUDA id and for cpu use -1
    task="text-generation",
    pipeline_kwargs={"temperature": 0.1, "max_new_tokens": 500}
)

llm.invoke(query)


Device set to use cuda:0


'What are the system log for (FAU_SAR)?  The system log for (FAU_SAR) is not available in the system log.  The system log for (FAU_SAR) is not available in the system log.\nThe system log for (FAU_SAR) is not available in the system log.  The system log for (FAU_SAR) is not available in the system log.\nThe system log for (FAU_SAR) is not available in the system log.  The system log for (FAU_SAR) is not available in the system log.\nThe system log for (FAU_SAR) is not available in the system log.  The system log for (FAU_SAR) is not available in the system log.\nThe system log for (FAU_SAR) is not available in the system log.  The system log for (FAU_SAR) is not available in the system log.\nThe system log for (FAU_SAR) is not available in the system log.  The system log for (FAU_SAR) is not available in the system log.\nThe system log for (FAU_SAR) is not available in the system log.  The system log for (FAU_SAR) is not available in the system log.\nThe system log for (FAU_SAR) is not

In [18]:
query = """Hallo Wie ghets?"""  # Sample question, change to other questions you are interested in.
llm.invoke(query)


'Hallo Wie ghets? Ich habe ein Problem mit meiner E-Mail-Adresse. Ich habe mich mit einem anderen Benutzer namens "John" verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke, ich habe ihn falsch verstanden. Ich habe mich mit ihm verabschiedet, aber ich denke

## 7) A German-aware Prompt

Steer the model to stay grounded in retrieved context and answer in German or english (medium of instruction of question).


In [19]:
prompt_template = """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer".
2. If you find the answer, write the answer in a concise way.
3. Question will be in either English or German Language. Strictly follow the medium of language as per the given query for answer.

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)


In [20]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": PROMPT}
)

In [21]:
retrievalQA(query)

/tmp/ipykernel_3530/2156977366.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrievalQA(query)


{'query': 'Hallo Wie ghets?',
 'result': 'Use the following pieces of context to answer the question at the end. Please follow the following rules:\n1. If you don\'t know the answer, don\'t try to make up an answer. Just say "I can\'t find the final answer".\n2. If you find the answer, write the answer in a concise way.\n3. Question will be in either English or German Language. Strictly follow the medium of language as per the given query for answer.\n\nstellen und Sicherheitsupdates in kleinen IT-Umgebungen dienen.\nName Ver-\nsion\nGefähr-\ndungs-\npotenzial\nWichtigkeit Offene \nSchwach-\nstellen?\nGefährdung Sicherheits-\nupdate \nverfügbar\nSicherheits\n-update \neingespielt\nMitigation letzte \nPrüfung\nAnmerkungen\nTextEditor 1.31 niedrig niedrig nein - - 17.11.2013 -\nBrowser A 31 hoch hoch ja hoch nein nein Browser B \nnutzen\n18.11.2013 CVE 2013-xxxx, \nRemote Code \nExecution mit \nBenutzerrechten\n\nfig auf. Diese Fehler müssen nicht immer unmittelbar sicherheitskritische F

In [22]:
def ask_question(query):
    qa_chain = RetrievalQA.from_llm(
        llm, retriever=vectorstore.as_retriever(), prompt=PROMPT,return_source_documents=False
    )
    out=qa_chain(query)["result"]
    out=out.split("Helpful Answer:")[-1].strip()

    return out

query="""What are the system log for (FAU_SAR)? """  
print(ask_question(query))

The system log for (FAU_SAR) is not available in the provided context.


## Testing

In [23]:
#German Query

query="""Was ist der Testfall, um die Fähigkeit des Objekts zur Durchführung der Werkseinstellung zu überprüfen?"""  
print(ask_question(query))

I can't find the final answer.


In [24]:
# short ambigous query

query="""MustDoFactoryResetClsAsClient Zweck"""  
print(ask_question(query))

MustDoFactoryResetClsAsClient ist ein Werkzeug zur Erstellung von Sicherheitsupdates, um die Sicherheit von MustDoFactoryReset- 
Produkten zu verbessern. Es ermöglicht es, die Sicherheitsupdates zu erstellen, die für MustDoFactoryReset- 
Produkte speziell entwickelt wurden, um die Sicherheit der Daten zu verbessern und die Gefährdungslage zu reduzieren. 

I can't find the final answer.


In [25]:
# English query

query="""give me the test case for Factory setting"""  
print(ask_question(query))

The test case for Factory setting is:
4.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.


In [26]:
## Checking on False cases

query="""Hallo wie ghets """  
print(ask_question(query))

I can't find the final answer. 

Note: The question is not clear, and the answer is not a numerical value. The question seems to be asking about the importance of patching security updates in small IT environments, but the answer is not a numerical value. The correct answer is "I can't find the final answer."


In [27]:

query="""Explain in brief, What this pdf Smart metering systmes is about?"""  
print(ask_question(query))

The Smart Metering System is about the management of energy consumption and the monitoring of energy usage in households and businesses. It is a system that uses advanced technologies such as sensors, data analytics, and machine learning to track and analyze energy usage patterns, identify potential energy-saving opportunities, and provide insights to consumers and utilities. The system is designed to optimize energy efficiency and reduce energy waste, while also providing a secure and reliable way to manage energy consumption.


## 8) Gradio UI

A simple gradio UI to test question answering on PDF's

In [32]:
!pip install gradio --quiet


In [35]:
import gradio as gr

gradio_ui = gr.Interface(
    fn=ask_question,
    inputs=gr.Textbox(lines=2, label="Enter your question"),
    outputs=gr.Textbox(label="RAG Answer"),
    title="RAG Pipeline QA",
    description="Database contains 2 docs: pp0073b.pdf and TR-03109-5_Testspezifikation_german.pdf"
)

# Launch the interface in the notebook
gradio_ui.launch(inline=True,share=True)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://edcdf3c63549ba7a03.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
